In [ ]:
import os
import json
import folium
from sqlalchemy import create_engine
from geoalchemy2 import Geometry
from geoalchemy2.functions import ST_BdMPolyFromText, ST_PolygonFromText
from geoalchemy2.elements import WKTElement
from llama_index import GPTSQLStructStoreIndex, SQLDatabase
from llama_index.indices.struct_store import SQLContextContainerBuilder
from langchain import OpenAI

In [ ]:
OPENAI_API_KEY = os.getenv("OPENAI_API_KEY")
DATABASE_NAME = os.getenv("DATABASE_NAME")
DATABASE_USER = os.getenv("DATABASE_USER")
DATABASE_PASSWORD = os.getenv("DATABASE_PASSWORD")
DATABASE_PORT = os.getenv("DATABASE_PORT")
DATABASE_HOST = os.getenv("DATABASE_HOST")

In [ ]:
llm_predictor = OpenAI(model_name="gpt-4")

In [ ]:
connection_string = f"postgresql://{DATABASE_USER}:{DATABASE_PASSWORD}@{DATABASE_HOST}:{DATABASE_PORT}/{DATABASE_NAME}"
engine = create_engine(connection_string)

In [ ]:
db_all = SQLDatabase(engine)

In [ ]:
index_all = GPTSQLStructStoreIndex.from_documents(
    [],
    sql_database=db_all,
)

In [ ]:
res = index_all.query("Get the names of all tables in database?")

In [ ]:
type([m[0] for m in res.extra_info["result"]]).__name__

In [ ]:
db_stand_4326 = SQLDatabase(engine, include_tables=["stand_4326"])

In [ ]:
stand_4326_text = (
    "The PostGis plugin is available in the database.\n"
    "The forest polygons can be found from the geometry column.\n"
    "Get the polygons as geojson."
)

table_context_dict = {
    "stand_4326": stand_4326_text,
}
context_builder = SQLContextContainerBuilder(db_stand_4326, context_dict=table_context_dict)
context_container = context_builder.build_context_container()

In [ ]:
index = GPTSQLStructStoreIndex.from_documents(
    [],
    sql_database=db_stand_4326,
    table_name="stand_4326",
    sql_context_container=context_container,
)

In [ ]:
res = index.query("Get the id and forest polygon of forest with larges area that has main tree species 2. Don't include NULL values.")

In [ ]:
id, pol = res.extra_info["result"][0]

In [ ]:
res.extra_info["sql_query"]

In [ ]:
id

In [ ]:
json.loads(pol)